In [24]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
from itertools import combinations

## Import Data

In [101]:
cap_gen_df = pd.read_csv('../data/EIA/monthly_state_capacity_generation.csv')
cap_gen_pr_df = pd.read_csv('../data/processed/gen_cap_reg_data.csv')

## Clean Data

In [26]:
merged_df = cap_gen_df.copy()

In [27]:
# Change column names
merged_df.rename(columns = {'Wind Cap': 'wind', 'Est Tot Solar Cap': 'solar', 'All Sectors Gen': 'all_sec_gen',
                           'Biomass Cap': 'biomass', 'Geothermal Cap': 'geo', 'Nuclear Cap': 'nuclear',
                           'Conventional Hydroelectric Cap': 'hydro', 'Renewable Sources Cap': 'renew', 
                           'Fossil Fuels Cap': 'fossil'}, 
                 inplace = True)

In [28]:
# Select only states (filter out regions)
states_list = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

merged_df = merged_df.query('State in @states_list').copy()

In [29]:
# Add month variable
merged_df['Month'] = merged_df['Date'].apply(lambda x: x.split(' ')[0])

## Export Data

In [30]:
merged_df.to_csv('../data/processed/capacity_reg_data.csv', index = False)

## Regressions

### Pairs

In [137]:
def convert_coefficients(coeffs):
    ''' Input coeffs consists of [delta*v, (1-delta)*v, -(1/2)*v*p*delta]
    log(Y) = (v/p)*log(delta*K^(-p) + (1-delta)*K^(-p))
    ~= log(Y) = v*delta*log(K) + v*(1-delta)*log(L) - (1/2)*p*v*delta*(1-delta)(log(K) - log(L))^2 
    where delta is the share parameter for the first input
          v is the scale parameter 
          p is the elasticity parameter (elas = 1 / (1+p))
    '''
    
    # delta / (1-delta)
    temp_1 = coeffs[0]/coeffs[1]
    delta = temp_1 / (temp_1 + 1)
    
    # v 
    v = coeffs[0] / delta
    
    # -(1/2)*v*p*delta  / (v*delta)
    temp_2 = coeffs[2] / coeffs[0]
    p = temp_2 / (-1/2)
    sigma = 1 / (1 + p)
    
    return [delta, v, sigma]

In [138]:
fits = {}

In [139]:
fits['wind_solar'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(solar+1) + np.log(wind+1) + np.power((np.log(solar+1) - np.log(wind+1)), 2)', 
        data = merged_df).fit()
fits['wind_solar'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.112
Model:                                 OLS   Adj. R-squared:                  0.110
Method:                      Least Squares   F-statistic:                     49.42
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):           4.47e-30
Time:                             12:44:26   Log-Likelihood:                -1674.0
No. Observations:                     1174   AIC:                             3356.
Df Residuals:                         1170   BIC:                             3376.
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                               7.4933      0.088     85.178      0.000       7.321       7.666
np.log(solar + 1)                                       0.1752      0.015     11.972      0.000       0.146       0.204
np.log(wind + 1)                                        0.0033      0.010      0.346      0.729      -0.016       0.022
np.power((np.log(solar + 1) - np.log(wind + 1)), 2)     0.0127      0.002      5.975      0.000       0.009       0.017
==============================================================================
Omnibus:                      101.769   Durbin-Watson:                   0.138
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              128.547
Skew:                          -0.751   Prob(JB):                     1.22e-28
Kurtosis:                       3.611   Cond. No.                         66.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [140]:
fits['biomass_nuclear'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(biomass+1) + np.log(nuclear+1) + np.power((np.log(biomass+1) - np.log(nuclear+1)), 2)', 
        data = merged_df).fit()
fits['biomass_nuclear'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.411
Model:                                 OLS   Adj. R-squared:                  0.409
Method:                      Least Squares   F-statistic:                     272.0
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):          6.72e-134
Time:                             12:44:27   Log-Likelihood:                -1433.5
No. Observations:                     1174   AIC:                             2875.
Df Residuals:                         1170   BIC:                             2895.
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
============================================================================================================================
                                                               coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    7.5072      0.066    114.000      0.000       7.378       7.636
np.log(biomass + 1)                                          0.0462      0.017      2.660      0.008       0.012       0.080
np.log(nuclear + 1)                                          0.1557      0.009     17.555      0.000       0.138       0.173
np.power((np.log(biomass + 1) - np.log(nuclear + 1)), 2)    -0.0073      0.003     -2.626      0.009      -0.013      -0.002
==============================================================================
Omnibus:                       46.783   Durbin-Watson:                   0.120
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.038
Skew:                          -0.479   Prob(JB):                     5.01e-12
Kurtosis:                       3.384   Cond. No.                         39.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [141]:
fits['geo_hydro'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(geo+1) + np.log(hydro+1) + np.power((np.log(geo+1) - np.log(hydro+1)), 2)', 
        data = merged_df).fit()
fits['geo_hydro'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.082
Model:                                 OLS   Adj. R-squared:                  0.080
Method:                      Least Squares   F-statistic:                     34.88
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):           1.37e-21
Time:                             12:44:27   Log-Likelihood:                -1693.8
No. Observations:                     1174   AIC:                             3396.
Df Residuals:                         1170   BIC:                             3416.
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              7.4405      0.108     68.865      0.000       7.229       7.652
np.log(geo + 1)                                       -0.1385      0.042     -3.335      0.001      -0.220      -0.057
np.log(hydro + 1)                                      0.2317      0.043      5.371      0.000       0.147       0.316
np.power((np.log(geo + 1) - np.log(hydro + 1)), 2)    -0.0101      0.005     -2.203      0.028      -0.019      -0.001
==============================================================================
Omnibus:                      131.597   Durbin-Watson:                   0.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              192.341
Skew:                          -0.817   Prob(JB):                     1.71e-42
Kurtosis:                       4.124   Cond. No.                         159.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [143]:
fits['rnw_ff'] = smf.ols('np.log(all_sec_gen+1) ~ np.log(renew+1) + np.log(fossil+1) + np.power((np.log(renew+1) - np.log(fossil+1)), 2)', 
        data = merged_df).fit()
fits['rnw_ff'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.925
Model:                                 OLS   Adj. R-squared:                  0.925
Method:                      Least Squares   F-statistic:                     4804.
Date:                     Thu, 03 Jan 2019   Prob (F-statistic):               0.00
Time:                             12:44:36   Log-Likelihood:                -224.21
No. Observations:                     1174   AIC:                             456.4
Df Residuals:                         1170   BIC:                             476.7
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
=========================================================================================================================
                                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                                -0.8410      0.082    -10.265      0.000      -1.002      -0.680
np.log(renew + 1)                                         0.4254      0.013     31.623      0.000       0.399       0.452
np.log(fossil + 1)                                        0.6283      0.011     59.108      0.000       0.607       0.649
np.power((np.log(renew + 1) - np.log(fossil + 1)), 2)     0.0645      0.004     17.552      0.000       0.057       0.072
==============================================================================
Omnibus:                       11.722   Durbin-Watson:                   0.461
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               14.374
Skew:                          -0.144   Prob(JB):                     0.000756
Kurtosis:                       3.459   Cond. No.                         124.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [144]:
results = []

for fit_type, fit_data in fits.items():
    
    results.append([fit_type] + convert_coefficients(list(fit_data.params[1:])))

In [145]:
pd.DataFrame(results, columns = ['pair', 'delta', 'scale', 'elasticity'])

,pair,delta,scale,elasticity
0,biomass_nuclear,0.228874,0.201970,0.759279
1,wind_solar,0.981347,0.178512,1.170320
2,geo_hydro,-1.486088,0.093211,1.170211
3,rnw_ff,0.403753,1.053686,1.434816


### All Renewables

In [17]:
## Construct terms for regression
renewables = ['solar', 'wind', 'geo', 'biomass', 'nuclear', 'hydro']

# Log of each source
beta_terms = ' + '.join(['np.log({0} + 1)'.format(x) for x in renewables])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['np.log({0} + 1)*np.log({1}+1)'.format(x, y) for x, y in combinations(renewables, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['np.power(np.log({0} + 1), 2)'.format(x) for x in renewables])

In [18]:
fit_renew = smf.ols('np.log(all_sec_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = merged_df).fit()

fit_renew.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     np.log(all_sec_gen + 1)   R-squared:                       0.742
Model:                                 OLS   Adj. R-squared:                  0.736
Method:                      Least Squares   F-statistic:                     122.1
Date:                     Sun, 27 Jan 2019   Prob (F-statistic):          3.54e-314
Time:                             09:54:22   Log-Likelihood:                -948.77
No. Observations:                     1174   AIC:                             1954.
Df Residuals:                         1146   BIC:                             2095.
Df Model:                               27                                         
Covariance Type:                 nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                   4.8106      0.376     12.791      0.000       4.073       5.548
np.log(solar + 1)                           0.1898      0.054      3.509      0.000       0.084       0.296
np.log(wind + 1)                           -0.0943      0.057     -1.652      0.099      -0.206       0.018
np.log(geo + 1)                            -3.2363      0.384     -8.427      0.000      -3.990      -2.483
np.log(biomass + 1)                         0.7467      0.120      6.204      0.000       0.511       0.983
np.log(nuclear + 1)                        -0.3481      0.049     -7.111      0.000      -0.444      -0.252
np.log(hydro + 1)                           0.0984      0.042      2.340      0.019       0.016       0.181
np.log(solar + 1):np.log(wind + 1)          0.0154      0.004      3.527      0.000       0.007       0.024
np.log(solar + 1):np.log(geo + 1)          -0.0243      0.017     -1.463      0.144      -0.057       0.008
np.log(solar + 1):np.log(biomass + 1)      -0.0580      0.010     -5.900      0.000      -0.077      -0.039
np.log(solar + 1):np.log(nuclear + 1)       0.0355      0.005      7.500      0.000       0.026       0.045
np.log(solar + 1):np.log(hydro + 1)        -0.0088      0.005     -1.737      0.083      -0.019       0.001
np.log(wind + 1):np.log(geo + 1)            0.4764      0.059      8.061      0.000       0.360       0.592
np.log(wind + 1):np.log(biomass + 1)       -0.0611      0.010     -5.865      0.000      -0.081      -0.041
np.log(wind + 1):np.log(nuclear + 1)        0.0042      0.003      1.229      0.219      -0.003       0.011
np.log(wind + 1):np.log(hydro + 1)         -0.0080      0.005     -1.514      0.130      -0.018       0.002
np.log(geo + 1):np.log(biomass + 1)         0.0018      0.025      0.073      0.942      -0.047       0.050
np.log(geo + 1):np.log(nuclear + 1)        -0.2496      0.030     -8.375      0.000      -0.308      -0.191
np.log(geo + 1):np.log(hydro + 1)          -0.1332      0.022     -6.067      0.000      -0.176      -0.090
np.log(biomass + 1):np.log(nuclear + 1)     0.0328      0.006      5.220      0.000       0.020       0.045
np.log(biomass + 1):np.log(hydro + 1)      -0.0261      0.012     -2.206      0.028      -0.049      -0.003
np.log(nuclear + 1):np.log(hydro + 1)       0.0014      0.005      0.299      0.765      -0.008       0.010
np.power(np.log(solar + 1), 2)             -0.0069      0.005     -1.319      0.188      -0.017       0.003
np.power(np.log(wind + 1), 2)               0.0566      0.003     18.260      0.000       0.051       0.063
np.power(np.log(geo + 1), 2)                0.3343      0.044      7.547      0.000       0.247       0.421
np.p

In [67]:
# Collect results from fit
results = []

for source in renewables:
    
    gamma = fit_renew.params['np.power(np.log({0} + 1), 2)'.format(source)]
    beta = fit_renew.params['np.log({0} + 1)'.format(source)]
    results.append([source, gamma, beta])

# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

results_df

#### Mixed Effects (State clusters)

In [104]:
fit_renew_mixed = smf.mixedlm('np.log(all_sec_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = merged_df, groups = merged_df["State"]).fit()
fit_renew_mixed.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Mixed Linear Model Regression Results
==================================================================================
Model:                 MixedLM     Dependent Variable:     np.log(all_sec_gen + 1)
No. Observations:      1174        Method:                 REML                   
No. Groups:            50          Scale:                  0.0231                 
Min. group size:       14          Likelihood:             314.5643               
Max. group size:       24          Converged:              Yes                    
Mean group size:       23.5                                                       
----------------------------------------------------------------------------------
                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------
Intercept                                6.621    1.211  5.466 0.000  4.247  8.995
np.log(solar + 1)                       -0.020    0.053 -0.376 0.707 -0.124  0.084
np.log(wind + 1)                         0.054    0.130  0.416 0.677 -0.200  0.308
np.log(geo + 1)                          0.521    0.745  0.700 0.484 -0.939  1.981
np.log(biomass + 1)                      0.026    0.358  0.073 0.942 -0.676  0.728
np.log(nuclear + 1)                     -0.155    0.161 -0.960 0.337 -0.471  0.161
np.log(hydro + 1)                        0.001    0.211  0.003 0.998 -0.412  0.414
np.log(solar + 1):np.log(wind + 1)      -0.005    0.003 -1.581 0.114 -0.011  0.001
np.log(solar + 1):np.log(geo + 1)        0.008    0.009  0.862 0.389 -0.010  0.026
np.log(solar + 1):np.log(biomass + 1)   -0.000    0.009 -0.027 0.979 -0.018  0.018
np.log(solar + 1):np.log(nuclear + 1)    0.003    0.004  0.798 0.425 -0.005  0.012
np.log(solar + 1):np.log(hydro + 1)     -0.008    0.006 -1.362 0.173 -0.021  0.004
np.log(wind + 1):np.log(geo + 1)        -0.231    0.095 -2.437 0.015 -0.418 -0.045
np.log(wind + 1):np.log(biomass + 1)    -0.015    0.025 -0.589 0.556 -0.063  0.034
np.log(wind + 1):np.log(nuclear + 1)    -0.010    0.011 -0.862 0.389 -0.031  0.012
np.log(wind + 1):np.log(hydro + 1)      -0.003    0.017 -0.181 0.856 -0.037  0.031
np.log(geo + 1):np.log(biomass + 1)      0.057    0.078  0.721 0.471 -0.097  0.210
np.log(geo + 1):np.log(nuclear + 1)     -0.001    0.069 -0.010 0.992 -0.137  0.135
np.log(geo + 1):np.log(hydro + 1)        0.126    0.078  1.618 0.106 -0.027  0.278
np.log(biomass + 1):np.log(nuclear + 1)  0.027    0.021  1.298 0.194 -0.014  0.068
np.log(biomass + 1):np.log(hydro + 1)    0.002    0.043  0.058 0.954 -0.081  0.086
np.log(nuclear + 1):np.log(hydro + 1)    0.004    0.021  0.172 0.864 -0.038  0.045
np.power(np.log(solar + 1), 2)           0.009    0.003  2.558 0.011  0.002  0.015
np.power(np.log(wind + 1), 2)            0.024    0.011  2.246 0.025  0.003  0.045
np.power(np.log(geo + 1), 2)            -0.013    0.090 -0.148 0.882 -0.189  0.162
np.power(np.log(biomass + 1), 2)        -0.010    0.027 -0.358 0.720 -0.063  0.043
np.power(np.log(nuclear + 1), 2)         0.033    0.015  2.115 0.034  0.002  0.063
np.power(np.log(hydro + 1), 2)           0.003    0.021  0.132 0.895 -0.039  0.045
Group Var                                0.606    0.988                           
==================================================================================

"""

In [105]:
# Collect results from fit
results = []

for source in renewables:
    
    gamma = fit_renew_mixed.params['np.power(np.log({0} + 1), 2)'.format(source)]
    beta = fit_renew_mixed.params['np.log({0} + 1)'.format(source)]
    results.append([source, gamma, beta])

In [106]:
# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

results_df

,source,gamma,beta,phi,elasticity
0,solar,0.008769,-0.019927,0.840865,0.543223
1,wind,0.023979,0.053932,-1.017774,-56.261123
2,geo,-0.013303,0.521074,-0.231578,1.301369
3,biomass,-0.009687,0.026116,0.790164,0.558608
4,nuclear,0.032625,-0.154834,0.309260,0.763790
5,hydro,0.002833,0.000577,-9.838597,-0.113140


#### Mixed Effects (Month Cluster)

In [122]:
fit_renew_mixed = smf.mixedlm('np.log(all_sec_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = merged_df, groups = merged_df["Month"]).fit()
fit_renew_mixed.summary()

/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Conve

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Mixed Linear Model Regression Results
==================================================================================
Model:                 MixedLM     Dependent Variable:     np.log(all_sec_gen + 1)
No. Observations:      1174        Method:                 REML                   
No. Groups:            12          Scale:                  0.2962                 
Min. group size:       92          Likelihood:             -1059.6335             
Max. group size:       100         Converged:              No                     
Mean group size:       97.8                                                       
----------------------------------------------------------------------------------
                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------
Intercept                                4.821    0.373 12.920 0.000  4.090  5.552
np.log(solar + 1)                        0.192    0.054  3.588 0.000  0.087  0.297
np.log(wind + 1)                        -0.094    0.057 -1.663 0.096 -0.205  0.017
np.log(geo + 1)                         -3.207    0.381 -8.418 0.000 -3.953 -2.460
np.log(biomass + 1)                      0.741    0.119  6.217 0.000  0.508  0.975
np.log(nuclear + 1)                     -0.344    0.049 -7.097 0.000 -0.440 -0.249
np.log(hydro + 1)                        0.096    0.042  2.307 0.021  0.014  0.178
np.log(solar + 1):np.log(wind + 1)       0.016    0.004  3.642 0.000  0.007  0.024
np.log(solar + 1):np.log(geo + 1)       -0.023    0.017 -1.381 0.167 -0.055  0.010
np.log(solar + 1):np.log(biomass + 1)   -0.057    0.010 -5.841 0.000 -0.076 -0.038
np.log(solar + 1):np.log(nuclear + 1)    0.036    0.005  7.644 0.000  0.027  0.045
np.log(solar + 1):np.log(hydro + 1)     -0.009    0.005 -1.743 0.081 -0.019  0.001
np.log(wind + 1):np.log(geo + 1)         0.473    0.059  8.062 0.000  0.358  0.587
np.log(wind + 1):np.log(biomass + 1)    -0.061    0.010 -5.882 0.000 -0.081 -0.040
np.log(wind + 1):np.log(nuclear + 1)     0.004    0.003  1.137 0.256 -0.003  0.010
np.log(wind + 1):np.log(hydro + 1)      -0.008    0.005 -1.606 0.108 -0.019  0.002
np.log(geo + 1):np.log(biomass + 1)      0.001    0.024  0.023 0.981 -0.047  0.048
np.log(geo + 1):np.log(nuclear + 1)     -0.247    0.030 -8.363 0.000 -0.305 -0.189
np.log(geo + 1):np.log(hydro + 1)       -0.132    0.022 -6.068 0.000 -0.175 -0.089
np.log(biomass + 1):np.log(nuclear + 1)  0.033    0.006  5.225 0.000  0.020  0.045
np.log(biomass + 1):np.log(hydro + 1)   -0.026    0.012 -2.253 0.024 -0.049 -0.003
np.log(nuclear + 1):np.log(hydro + 1)    0.001    0.004  0.292 0.771 -0.008  0.010
np.power(np.log(solar + 1), 2)          -0.008    0.005 -1.570 0.116 -0.018  0.002
np.power(np.log(wind + 1), 2)            0.057    0.003 18.445 0.000  0.051  0.063
np.power(np.log(geo + 1), 2)             0.331    0.044  7.513 0.000  0.244  0.417
np.power(np.log(biomass + 1), 2)        -0.020    0.009 -2.337 0.019 -0.037 -0.003
np.power(np.log(nuclear + 1), 2)         0.021    0.005  4.441 0.000  0.012  0.031
np.power(np.log(hydro + 1), 2)           0.010    0.005  1.815 0.069 -0.001  0.020
Group Var                                0.005    0.006                           
==================================================================================

"""

In [125]:
# Collect results from fit
results = []

for source in renewables:
    
    gamma = fit_renew_mixed.params['np.power(np.log({0} + 1), 2)'.format(source)]
    beta = fit_renew_mixed.params['np.log({0} + 1)'.format(source)]
    results.append([source, gamma, beta])

In [126]:
# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

results_df

,source,gamma,beta,phi,elasticity
0,solar,-0.008192,0.192377,0.079331,0.926500
1,wind,0.056610,-0.094014,1.249189,0.444605
2,geo,0.330542,-3.206802,-0.911599,11.312042
3,biomass,-0.020318,0.741402,0.042704,0.959045
4,nuclear,0.021450,-0.344468,0.143432,0.874560
5,hydro,0.009662,0.096145,-0.193858,1.240476


## Generation Regs

### On Price with Generation

In [156]:
## Construct terms for regression
sources = ['solar', 'wind', 'geothermal', 'biomass', 'nuclear', 'hydro', 'fossil']

# Log of each source
beta_terms = ' + '.join(['ln_{0}_gen'.format(x) for x in sources])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['ln_{0}_{1}_gen'.format(x, y) for x, y in combinations(sources, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['ln_{0}_{0}_gen'.format(x) for x in sources])

In [157]:
fit_all = smf.ols('np.log(all_sectors_price+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = cap_gen_pr_df).fit()

fit_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     np.log(all_sectors_price + 1)   R-squared:                       0.744
Model:                                       OLS   Adj. R-squared:                  0.734
Method:                            Least Squares   F-statistic:                     70.24
Date:                           Mon, 04 Feb 2019   Prob (F-statistic):          2.48e-223
Time:                                   19:37:31   Log-Likelihood:                 577.68
No. Observations:                            880   AIC:                            -1083.
Df Residuals:                                844   BIC:                            -911.3
Df Model:                                     35                                         
Covariance Type:                       nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        2.6610      0.123     21.721      0.000       2.421       2.902
ln_solar_gen                     0.0314      0.010      3.235      0.001       0.012       0.050
ln_wind_gen                     -0.0183      0.012     -1.511      0.131      -0.042       0.005
ln_geothermal_gen               -0.6285      0.067     -9.415      0.000      -0.760      -0.497
ln_biomass_gen                  -0.1026      0.015     -6.642      0.000      -0.133      -0.072
ln_nuclear_gen                   0.0406      0.011      3.640      0.000       0.019       0.063
ln_hydro_gen                     0.1478      0.010     14.442      0.000       0.128       0.168
ln_fossil_gen                   -0.0114      0.008     -1.430      0.153      -0.027       0.004
ln_solar_wind_gen               -0.0005      0.000     -1.695      0.090      -0.001    7.96e-05
ln_solar_geothermal_gen         -0.0008      0.001     -0.936      0.349      -0.002       0.001
ln_solar_biomass_gen            -0.0005      0.000     -1.337      0.182      -0.001       0.000
ln_solar_nuclear_gen            -0.0003      0.000     -1.066      0.287      -0.001       0.000
ln_solar_hydro_gen              -0.0035      0.001     -5.024      0.000      -0.005      -0.002
ln_solar_fossil_gen          -6.575e-05      0.000     -0.183      0.855      -0.001       0.001
ln_wind_geothermal_gen           0.0046      0.003      1.537      0.125      -0.001       0.010
ln_wind_biomass_gen              0.0081      0.001     12.170      0.000       0.007       0.009
ln_wind_nuclear_gen             -0.0002      0.000     -1.069      0.285      -0.001       0.000
ln_wind_hydro_gen                0.0002      0.001      0.282      0.778      -0.001       0.002
ln_wind_fossil_gen               0.0018      0.000      5.583      0.000       0.001       0.002
ln_geothermal_biomass_gen        0.0454      0.003     13.027      0.000       0.039       0.052
ln_geothermal_nuclear_gen       -0.0120      0.001     -8.538      0.000      -0.015      -0.009
ln_geothermal_hydro_gen         -0.0085      0.001     -6.434      0.000      -0.011      -0.006
ln_geothermal_fossil_gen         0.0008      0.000      1.817      0.070   -6.57e-05       0.002
ln_biomass_nuclear_gen          -0.0004      0.000     -1.472      0.141      -0.001       0.000
ln_biomass_hydro_gen            -0.0027      0.001     -3.067      0.002      -0.004      -0.001
ln_biomass_fossil_gen           -0.0005      0.000     -1.056      0.291      -0.001       0.000
ln_nuclear_hydro_gen            -0.0021      0.000     -4.674      0.000      -0.003      -0.001
ln_nuclear_fossil_gen            0.0007      0.000  

In [158]:
# Collect results from fit
results = []

for source in sources:
    
    gamma = fit_all.params['ln_{0}_{0}_gen'.format(source)]
    beta = fit_all.params['ln_{0}_gen'.format(source)]
    results.append([source, gamma, beta])

# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

display(results_df)
print('Average elasticity: {0:.5f}'.format(results_df['elasticity'].mean()))

,source,gamma,beta,phi,elasticity
0,solar,0.002474,0.031355,-0.149170,1.175323
1,wind,-0.006353,-0.018317,-0.717930,3.545216
2,geothermal,0.024031,-0.628509,-0.473545,1.899497
3,biomass,0.004888,-0.102599,0.117569,0.894799
4,nuclear,-0.001362,0.040624,0.062365,0.941296
5,hydro,-0.005754,0.147754,0.061183,0.942345
6,fossil,-0.000350,-0.011433,-0.062556,1.066730


Average elasticity: 1.49503


### On Gen with Generation

In [159]:
## Construct terms for regression
sources = ['solar', 'wind', 'geothermal', 'biomass', 'nuclear', 'hydro', 'fossil']

# Log of each source
beta_terms = ' + '.join(['ln_{0}_gen'.format(x) for x in sources])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['ln_{0}_{1}_gen'.format(x, y) for x, y in combinations(sources, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['ln_{0}_{0}_gen'.format(x) for x in sources])

In [160]:
fit_all = smf.ols('np.log(all_sectors_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = cap_gen_pr_df).fit()

fit_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     np.log(all_sectors_gen + 1)   R-squared:                       0.812
Model:                                     OLS   Adj. R-squared:                  0.804
Method:                          Least Squares   F-statistic:                     104.0
Date:                         Mon, 04 Feb 2019   Prob (F-statistic):          9.42e-279
Time:                                 19:37:52   Log-Likelihood:                -577.32
No. Observations:                          880   AIC:                             1227.
Df Residuals:                              844   BIC:                             1399.
Df Model:                                   35                                         
Covariance Type:                     nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       14.4867      0.455     31.826      0.000      13.593      15.380
ln_solar_gen                    -0.0120      0.036     -0.334      0.739      -0.083       0.059
ln_wind_gen                     -0.3039      0.045     -6.749      0.000      -0.392      -0.216
ln_geothermal_gen                0.1154      0.248      0.465      0.642      -0.371       0.602
ln_biomass_gen                   0.0380      0.057      0.662      0.508      -0.075       0.151
ln_nuclear_gen                  -0.1603      0.041     -3.865      0.000      -0.242      -0.079
ln_hydro_gen                    -0.2519      0.038     -6.628      0.000      -0.327      -0.177
ln_fossil_gen                    0.0664      0.030      2.235      0.026       0.008       0.125
ln_solar_wind_gen                0.0030      0.001      2.723      0.007       0.001       0.005
ln_solar_geothermal_gen          0.0097      0.003      3.087      0.002       0.004       0.016
ln_solar_biomass_gen            -0.0025      0.002     -1.641      0.101      -0.005       0.000
ln_solar_nuclear_gen             0.0062      0.001      6.351      0.000       0.004       0.008
ln_solar_hydro_gen              -0.0054      0.003     -2.109      0.035      -0.010      -0.000
ln_solar_fossil_gen              0.0051      0.001      3.814      0.000       0.002       0.008
ln_wind_geothermal_gen          -0.0132      0.011     -1.195      0.232      -0.035       0.008
ln_wind_biomass_gen             -0.0151      0.002     -6.080      0.000      -0.020      -0.010
ln_wind_nuclear_gen              0.0015      0.001      1.888      0.059   -6.04e-05       0.003
ln_wind_hydro_gen                0.0106      0.003      4.229      0.000       0.006       0.016
ln_wind_fossil_gen              -0.0037      0.001     -3.168      0.002      -0.006      -0.001
ln_geothermal_biomass_gen       -0.0031      0.013     -0.238      0.812      -0.028       0.022
ln_geothermal_nuclear_gen       -0.0091      0.005     -1.750      0.080      -0.019       0.001
ln_geothermal_hydro_gen          0.0125      0.005      2.554      0.011       0.003       0.022
ln_geothermal_fossil_gen        -0.0124      0.002     -7.422      0.000      -0.016      -0.009
ln_biomass_nuclear_gen           0.0053      0.001      5.485      0.000       0.003       0.007
ln_biomass_hydro_gen             0.0111      0.003      3.436      0.001       0.005       0.017
ln_biomass_fossil_gen            0.0015      0.002      0.814      0.416      -0.002       0.005
ln_nuclear_hydro_gen             0.0021      0.002      1.273      0.203      -0.001       0.005
ln_nuclear_fossil_gen           -0.0097      0.001    -11.013      0.000  

In [162]:
# Collect results from fit
results = []

for source in sources:
    
    gamma = fit_all.params['ln_{0}_{0}_gen'.format(source)]
    beta = fit_all.params['ln_{0}_gen'.format(source)]
    results.append([source, gamma, beta])

# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

display(results_df)
print('Average elasticity: {0:.5f}'.format(results_df['elasticity'].mean()))

,source,gamma,beta,phi,elasticity
0,solar,-0.000135,-0.012016,-0.022976,1.023517
1,wind,0.026422,-0.303924,0.432375,0.698141
2,geothermal,0.001741,0.115437,-0.024587,1.025206
3,biomass,-0.003632,0.038002,0.177858,0.848998
4,nuclear,0.016220,-0.160269,0.295605,0.771840
5,hydro,0.003290,-0.251948,0.051777,0.950772
6,fossil,-0.000351,0.066374,0.009348,0.990739


Average elasticity: 0.90132


### On Price with Capacity

In [111]:
## Construct terms for regression
sources = ['solar', 'wind', 'geothermal', 'biomass', 'nuclear', 'hydro', 'fossil']

# Log of each source
beta_terms = ' + '.join(['ln_{0}_cap'.format(x) for x in sources])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['ln_{0}_{1}_cap'.format(x, y) for x, y in combinations(sources, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['ln_{0}_{0}_cap'.format(x) for x in sources])

In [112]:
fit_all = smf.ols('np.log(all_sectors_price+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = cap_gen_pr_df).fit()

fit_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     np.log(all_sectors_price + 1)   R-squared:                       0.874
Model:                                       OLS   Adj. R-squared:                  0.868
Method:                            Least Squares   F-statistic:                     166.7
Date:                           Mon, 04 Feb 2019   Prob (F-statistic):               0.00
Time:                                   19:22:28   Log-Likelihood:                 887.54
No. Observations:                            880   AIC:                            -1703.
Df Residuals:                                844   BIC:                            -1531.
Df Model:                                     35                                         
Covariance Type:                       nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        7.7107      0.514     15.014      0.000       6.703       8.719
ln_solar_cap                    -0.0508      0.040     -1.266      0.206      -0.129       0.028
ln_wind_cap                     -0.1621      0.027     -5.984      0.000      -0.215      -0.109
ln_geothermal_cap               -0.1543      0.102     -1.516      0.130      -0.354       0.045
ln_biomass_cap                  -1.1481      0.101    -11.338      0.000      -1.347      -0.949
ln_nuclear_cap                   0.4242      0.041     10.320      0.000       0.344       0.505
ln_hydro_cap                     0.0720      0.057      1.263      0.207      -0.040       0.184
ln_fossil_cap                   -0.5274      0.094     -5.610      0.000      -0.712      -0.343
ln_solar_wind_cap                0.0068      0.001      5.756      0.000       0.004       0.009
ln_solar_geothermal_cap         -0.0133      0.004     -3.547      0.000      -0.021      -0.006
ln_solar_biomass_cap            -0.0190      0.003     -7.105      0.000      -0.024      -0.014
ln_solar_nuclear_cap             0.0078      0.001      7.959      0.000       0.006       0.010
ln_solar_hydro_cap              -0.0058      0.002     -2.917      0.004      -0.010      -0.002
ln_solar_fossil_cap              0.0068      0.005      1.333      0.183      -0.003       0.017
ln_wind_geothermal_cap          -0.0456      0.058     -0.792      0.429      -0.159       0.067
ln_wind_biomass_cap              0.0126      0.004      3.204      0.001       0.005       0.020
ln_wind_nuclear_cap             -0.0039      0.001     -3.663      0.000      -0.006      -0.002
ln_wind_hydro_cap                0.0008      0.001      0.736      0.462      -0.001       0.003
ln_wind_fossil_cap               0.0173      0.004      4.728      0.000       0.010       0.024
ln_geothermal_biomass_cap        0.0914      0.016      5.695      0.000       0.060       0.123
ln_geothermal_nuclear_cap       -0.0424      0.011     -3.811      0.000      -0.064      -0.021
ln_geothermal_hydro_cap         -0.0353      0.018     -1.963      0.050      -0.071   -2.04e-06
ln_geothermal_fossil_cap         0.0085      0.040      0.212      0.832      -0.070       0.088
ln_biomass_nuclear_cap          -0.0107      0.002     -5.454      0.000      -0.015      -0.007
ln_biomass_hydro_cap             0.0363      0.003     10.747      0.000       0.030       0.043
ln_biomass_fossil_cap            0.0890      0.008     10.952      0.000       0.073       0.105
ln_nuclear_hydro_cap            -0.0186      0.002    -12.191      0.000      -0.022      -0.016
ln_nuclear_fossil_cap           -0.0328      0.004  

In [113]:
# Collect results from fit
results = []

for source in sources:
    
    gamma = fit_all.params['ln_{0}_{0}_cap'.format(source)]
    beta = fit_all.params['ln_{0}_cap'.format(source)]
    results.append([source, gamma, beta])

# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

display(results_df)
print('Average elasticity: {0:.5f}'.format(results_df['elasticity'].mean()))

,source,gamma,beta,phi,elasticity
0,solar,0.012364,-0.050754,0.503763,0.664999
1,wind,-0.010332,-0.162096,-0.142401,1.166046
2,geothermal,0.073646,-0.154283,1.060489,0.485322
3,biomass,0.023863,-1.148086,0.161392,0.861035
4,nuclear,0.004892,0.424191,-0.018100,1.018434
5,hydro,-0.012068,0.072009,0.320279,0.757415
6,fossil,-0.000561,-0.527378,-0.003227,1.003238


Average elasticity: 0.85093


### On Gen with Capacity

In [114]:
## Construct terms for regression
sources = ['solar', 'wind', 'geothermal', 'biomass', 'nuclear', 'hydro', 'fossil']

# Log of each source
beta_terms = ' + '.join(['ln_{0}_cap'.format(x) for x in sources])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['ln_{0}_{1}_cap'.format(x, y) for x, y in combinations(sources, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['ln_{0}_{0}_cap'.format(x) for x in sources])

In [115]:
fit_all = smf.ols('np.log(all_sectors_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = cap_gen_pr_df).fit()

fit_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     np.log(all_sectors_gen + 1)   R-squared:                       0.976
Model:                                     OLS   Adj. R-squared:                  0.975
Method:                          Least Squares   F-statistic:                     979.4
Date:                         Mon, 04 Feb 2019   Prob (F-statistic):               0.00
Time:                                 19:22:29   Log-Likelihood:                 328.39
No. Observations:                          880   AIC:                            -584.8
Df Residuals:                              844   BIC:                            -412.7
Df Model:                                   35                                         
Covariance Type:                     nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        8.8857      0.970      9.165      0.000       6.983      10.789
ln_solar_cap                     0.1295      0.076      1.712      0.087      -0.019       0.278
ln_wind_cap                      0.0632      0.051      1.236      0.217      -0.037       0.164
ln_geothermal_cap                0.0981      0.192      0.511      0.610      -0.279       0.475
ln_biomass_cap                   0.3286      0.191      1.719      0.086      -0.047       0.704
ln_nuclear_cap                   0.4274      0.078      5.508      0.000       0.275       0.580
ln_hydro_cap                     0.3903      0.108      3.627      0.000       0.179       0.602
ln_fossil_cap                   -0.4273      0.177     -2.408      0.016      -0.776      -0.079
ln_solar_wind_cap               -0.0040      0.002     -1.795      0.073      -0.008       0.000
ln_solar_geothermal_cap         -0.0051      0.007     -0.720      0.472      -0.019       0.009
ln_solar_biomass_cap            -0.0011      0.005     -0.219      0.826      -0.011       0.009
ln_solar_nuclear_cap            -0.0090      0.002     -4.857      0.000      -0.013      -0.005
ln_solar_hydro_cap              -0.0173      0.004     -4.630      0.000      -0.025      -0.010
ln_solar_fossil_cap              0.0098      0.010      1.020      0.308      -0.009       0.029
ln_wind_geothermal_cap          -0.1454      0.109     -1.338      0.181      -0.359       0.068
ln_wind_biomass_cap              0.0004      0.007      0.058      0.954      -0.014       0.015
ln_wind_nuclear_cap              0.0034      0.002      1.687      0.092      -0.001       0.007
ln_wind_hydro_cap                0.0033      0.002      1.593      0.112      -0.001       0.007
ln_wind_fossil_cap              -0.0163      0.007     -2.357      0.019      -0.030      -0.003
ln_geothermal_biomass_cap        0.0542      0.030      1.790      0.074      -0.005       0.114
ln_geothermal_nuclear_cap        0.0299      0.021      1.424      0.155      -0.011       0.071
ln_geothermal_hydro_cap          0.0433      0.034      1.275      0.203      -0.023       0.110
ln_geothermal_fossil_cap         0.1214      0.076      1.597      0.111      -0.028       0.270
ln_biomass_nuclear_cap           0.0089      0.004      2.398      0.017       0.002       0.016
ln_biomass_hydro_cap            -0.0212      0.006     -3.320      0.001      -0.034      -0.009
ln_biomass_fossil_cap           -0.0650      0.015     -4.238      0.000      -0.095      -0.035
ln_nuclear_hydro_cap            -0.0022      0.003     -0.775      0.439      -0.008       0.003
ln_nuclear_fossil_cap           -0.0623      0.007     -8.516      0.000  

In [116]:
# Collect results from fit
results = []

for source in sources:
    
    gamma = fit_all.params['ln_{0}_{0}_cap'.format(source)]
    beta = fit_all.params['ln_{0}_cap'.format(source)]
    results.append([source, gamma, beta])

# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

display(results_df)
print('Average elasticity: {0:.5f}'.format(results_df['elasticity'].mean()))

,source,gamma,beta,phi,elasticity
0,solar,-0.006137,0.129479,0.108735,0.901929
1,wind,0.011644,0.063193,-0.393113,1.647754
2,geothermal,-0.141430,0.098069,3.194579,0.238403
3,biomass,0.034726,0.328552,-0.313350,1.456345
4,nuclear,0.026501,0.427362,-0.215028,1.273931
5,hydro,0.025300,0.390339,-0.211320,1.267941
6,fossil,0.115839,-0.427256,0.381021,0.724102


Average elasticity: 1.07292


### On Price with Gen Hat

In [117]:
## Construct terms for regression
sources = ['solar', 'wind', 'geothermal', 'biomass', 'nuclear', 'hydro', 'fossil']

# Log of each source
beta_terms = ' + '.join(['ln_{0}_gen_hat'.format(x) for x in sources])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['ln_{0}_{1}_gen_hat'.format(x, y) for x, y in combinations(sources, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['ln_{0}_{0}_gen_hat'.format(x) for x in sources])

In [118]:
fit_all = smf.ols('np.log(all_sectors_price+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = cap_gen_pr_df).fit()

fit_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     np.log(all_sectors_price + 1)   R-squared:                       0.885
Model:                                       OLS   Adj. R-squared:                  0.880
Method:                            Least Squares   F-statistic:                     185.2
Date:                           Mon, 04 Feb 2019   Prob (F-statistic):               0.00
Time:                                   19:22:29   Log-Likelihood:                 928.21
No. Observations:                            880   AIC:                            -1784.
Df Residuals:                                844   BIC:                            -1612.
Df Model:                                     35                                         
Covariance Type:                       nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                          119.1576     20.419      5.836      0.000      79.079     159.236
ln_solar_gen_hat                     0.4832      0.047     10.266      0.000       0.391       0.576
ln_wind_gen_hat                     -0.6161      1.000     -0.616      0.538      -2.580       1.348
ln_geothermal_gen_hat               -8.6936      1.989     -4.371      0.000     -12.597      -4.790
ln_biomass_gen_hat                  -5.2469      0.763     -6.875      0.000      -6.745      -3.749
ln_nuclear_gen_hat                   1.1306      0.163      6.917      0.000       0.810       1.451
ln_hydro_gen_hat                     0.5439      0.190      2.855      0.004       0.170       0.918
ln_fossil_gen_hat                   -7.4990      2.136     -3.510      0.000     -11.692      -3.306
ln_solar_wind_gen_hat               -0.0005      0.000     -2.195      0.028      -0.001   -4.88e-05
ln_solar_geothermal_gen_hat         -0.0164      0.003     -5.974      0.000      -0.022      -0.011
ln_solar_biomass_gen_hat            -0.0081      0.001     -5.833      0.000      -0.011      -0.005
ln_solar_nuclear_gen_hat             0.0009      0.000      3.281      0.001       0.000       0.001
ln_solar_hydro_gen_hat               0.0004      0.000      0.986      0.325      -0.000       0.001
ln_solar_fossil_gen_hat             -0.0196      0.003     -7.055      0.000      -0.025      -0.014
ln_wind_geothermal_gen_hat           0.0270      0.109      0.248      0.804      -0.187       0.241
ln_wind_biomass_gen_hat              0.0091      0.002      4.335      0.000       0.005       0.013
ln_wind_nuclear_gen_hat             -0.0006      0.000     -2.201      0.028      -0.001   -6.72e-05
ln_wind_hydro_gen_hat               -0.0019      0.000     -6.809      0.000      -0.002      -0.001
ln_wind_fossil_gen_hat               0.0277      0.003      7.926      0.000       0.021       0.035
ln_geothermal_biomass_gen_hat        0.1915      0.076      2.535      0.011       0.043       0.340
ln_geothermal_nuclear_gen_hat       -0.0801      0.018     -4.450      0.000      -0.115      -0.045
ln_geothermal_hydro_gen_hat         -0.0474      0.018     -2.649      0.008      -0.083      -0.012
ln_geothermal_fossil_gen_hat         0.2933      0.225      1.301      0.193      -0.149       0.736
ln_biomass_nuclear_gen_hat          -0.0072      0.001     -6.692      0.000      -0.009      -0.005
ln_biomass_hydro_gen_hat            -0.0013      0.002     -0.527      0.598      -0.006       0.003
ln_biomass_fossil_gen_hat            0.1888      0.023      8.137      0.000       0.143       0.234
ln_nuclear_hydro_gen_hat         

In [119]:
# Collect results from fit
results = []

for source in sources:
    
    gamma = fit_all.params['ln_{0}_{0}_gen_hat'.format(source)]
    beta = fit_all.params['ln_{0}_gen_hat'.format(source)]
    results.append([source, gamma, beta])

# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

display(results_df)
print('Average elasticity: {0:.5f}'.format(results_df['elasticity'].mean()))

,source,gamma,beta,phi,elasticity
0,solar,0.002420,0.483173,-0.009779,1.009876
1,wind,-0.006139,-0.616069,-0.020565,1.020997
2,geothermal,0.165158,-8.693567,0.067477,0.936789
3,biomass,0.051219,-5.246876,0.026516,0.974169
4,nuclear,0.000244,1.130583,-0.000408,1.000408
5,hydro,-0.001786,0.543858,0.006394,0.993646
6,fossil,0.098883,-7.498971,0.042323,0.959396


Average elasticity: 0.98504


### On Gen with Gen Hat

In [120]:
## Construct terms for regression
sources = ['solar', 'wind', 'geothermal', 'biomass', 'nuclear', 'hydro', 'fossil']

# Log of each source
beta_terms = ' + '.join(['ln_{0}_gen_hat'.format(x) for x in sources])

# Log source_i * Log source_j
gamma_ij_terms = ' + '.join(['ln_{0}_{1}_gen_hat'.format(x, y) for x, y in combinations(sources, 2)])

# ( Log source_i )^2
gamma_jj_terms = ' + '.join(['ln_{0}_{0}_gen_hat'.format(x) for x in sources])

In [121]:
fit_all = smf.ols('np.log(all_sectors_gen+1) ~ ' + ' + '.join([beta_terms, gamma_ij_terms, gamma_jj_terms]), 
        data = cap_gen_pr_df).fit()

fit_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     np.log(all_sectors_gen + 1)   R-squared:                       0.967
Model:                                     OLS   Adj. R-squared:                  0.965
Method:                          Least Squares   F-statistic:                     702.0
Date:                         Mon, 04 Feb 2019   Prob (F-statistic):               0.00
Time:                                 19:22:30   Log-Likelihood:                 186.00
No. Observations:                          880   AIC:                            -300.0
Df Residuals:                              844   BIC:                            -127.9
Df Model:                                   35                                         
Covariance Type:                     nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         -267.3505     47.460     -5.633      0.000    -360.505    -174.196
ln_solar_gen_hat                    -0.2832      0.109     -2.589      0.010      -0.498      -0.069
ln_wind_gen_hat                      1.9933      2.325      0.857      0.392      -2.571       6.557
ln_geothermal_gen_hat               22.1313      4.623      4.787      0.000      13.058      31.205
ln_biomass_gen_hat                   0.9957      1.774      0.561      0.575      -2.486       4.477
ln_nuclear_gen_hat                  -0.6964      0.380     -1.833      0.067      -1.442       0.049
ln_hydro_gen_hat                    -0.1063      0.443     -0.240      0.810      -0.975       0.763
ln_fossil_gen_hat                   22.0465      4.966      4.440      0.000      12.300      31.793
ln_solar_wind_gen_hat                0.0023      0.000      4.725      0.000       0.001       0.003
ln_solar_geothermal_gen_hat          0.0090      0.006      1.408      0.160      -0.004       0.021
ln_solar_biomass_gen_hat             0.0188      0.003      5.810      0.000       0.012       0.025
ln_solar_nuclear_gen_hat            -0.0005      0.001     -0.884      0.377      -0.002       0.001
ln_solar_hydro_gen_hat              -0.0065      0.001     -7.859      0.000      -0.008      -0.005
ln_solar_fossil_gen_hat              0.0016      0.006      0.250      0.802      -0.011       0.014
ln_wind_geothermal_gen_hat          -0.2179      0.253     -0.861      0.389      -0.715       0.279
ln_wind_biomass_gen_hat              0.0382      0.005      7.803      0.000       0.029       0.048
ln_wind_nuclear_gen_hat             -0.0045      0.001     -6.934      0.000      -0.006      -0.003
ln_wind_hydro_gen_hat                0.0032      0.001      4.831      0.000       0.002       0.004
ln_wind_fossil_gen_hat              -0.0401      0.008     -4.946      0.000      -0.056      -0.024
ln_geothermal_biomass_gen_hat        0.2342      0.176      1.334      0.182      -0.110       0.579
ln_geothermal_nuclear_gen_hat        0.0989      0.042      2.364      0.018       0.017       0.181
ln_geothermal_hydro_gen_hat          0.0547      0.042      1.316      0.188      -0.027       0.136
ln_geothermal_fossil_gen_hat        -1.2451      0.524     -2.377      0.018      -2.273      -0.217
ln_biomass_nuclear_gen_hat           0.0281      0.003     11.177      0.000       0.023       0.033
ln_biomass_hydro_gen_hat             0.0212      0.006      3.775      0.000       0.010       0.032
ln_biomass_fossil_gen_hat           -0.5297      0.054     -9.824      0.000      -0.636      -0.424
ln_nuclear_hydro_gen_hat            -0.0031      0.000 

In [122]:
# Collect results from fit
results = []

for source in sources:
    
    gamma = fit_all.params['ln_{0}_{0}_gen_hat'.format(source)]
    beta = fit_all.params['ln_{0}_gen_hat'.format(source)]
    results.append([source, gamma, beta])

# Convert results to elasticity estimates
results_df = pd.DataFrame(results, columns = ['source', 'gamma', 'beta'])

v = results_df['beta'].sum()
results_df['phi'] = results_df.apply(lambda x: 2*x.gamma / ((x.beta**2 / v) - x.beta), axis = 1)

results_df['elasticity'] = np.divide(1, np.add(1, results_df['phi']))

display(results_df)
print('Average elasticity: {0:.5f}'.format(results_df['elasticity'].mean()))

,source,gamma,beta,phi,elasticity
0,solar,0.003925,-0.283244,0.027547,0.973191
1,wind,0.013257,1.993261,-0.013903,1.014099
2,geothermal,-0.277899,22.131314,0.048321,0.953907
3,biomass,0.143758,0.995740,-0.295124,1.418689
4,nuclear,0.009884,-0.696405,0.027962,0.972799
5,hydro,0.004060,-0.106322,0.076194,0.929200
6,fossil,-0.090122,22.046509,0.015675,0.984567


Average elasticity: 1.03521


## Scratch

In [155]:
for reg_y in ['ln_all_sec_gen', 'ln_all_sec_pr']:
    for source_type in ['gen', 'cap', 'gen_hat']:
        
        print('// {0} on {1}'.format(reg_y, source_type))
        print('nl ({1} = {{c=1}} + ({0})^(1/{{phi=1}})), hasconstant(c) iterate(200) nolog'.format(
            ' + '.join(['exp({{ln_alpha_{0}=0}})*(ln_{0}_{1}^{{phi}})'.format(x, source_type) for x in sources]), reg_y))
        print('est sto {0}_{1}'.format(reg_y, source_type))
        print()

// ln_all_sec_gen on gen
nl (ln_all_sec_gen = {c=1} + (exp({ln_alpha_solar=0})*(ln_solar_gen^{phi}) + exp({ln_alpha_wind=0})*(ln_wind_gen^{phi}) + exp({ln_alpha_geothermal=0})*(ln_geothermal_gen^{phi}) + exp({ln_alpha_biomass=0})*(ln_biomass_gen^{phi}) + exp({ln_alpha_nuclear=0})*(ln_nuclear_gen^{phi}) + exp({ln_alpha_hydro=0})*(ln_hydro_gen^{phi}) + exp({ln_alpha_fossil=0})*(ln_fossil_gen^{phi}))^(1/{phi=1})), hasconstant(c) iterate(200) nolog
est sto ln_all_sec_gen_gen

// ln_all_sec_gen on cap
nl (ln_all_sec_gen = {c=1} + (exp({ln_alpha_solar=0})*(ln_solar_cap^{phi}) + exp({ln_alpha_wind=0})*(ln_wind_cap^{phi}) + exp({ln_alpha_geothermal=0})*(ln_geothermal_cap^{phi}) + exp({ln_alpha_biomass=0})*(ln_biomass_cap^{phi}) + exp({ln_alpha_nuclear=0})*(ln_nuclear_cap^{phi}) + exp({ln_alpha_hydro=0})*(ln_hydro_cap^{phi}) + exp({ln_alpha_fossil=0})*(ln_fossil_cap^{phi}))^(1/{phi=1})), hasconstant(c) iterate(200) nolog
est sto ln_all_sec_gen_cap

// ln_all_sec_gen on gen_hat
nl (ln_all_sec_ge

In [123]:
'(ln_all_sec_gen = {{c=1}} + ({0})^(1/{{phi=1}}))'.format(
    ' + '.join(['exp({{ln_alpha_{0}=2}})*(ln_{0}_cap^{{phi}})'.format(x) for x in sources]))

'(ln_all_sec_gen = {c=1} + (exp({ln_alpha_solar=2})*(ln_solar_cap^{phi}) + exp({ln_alpha_wind=2})*(ln_wind_cap^{phi}) + exp({ln_alpha_geothermal=2})*(ln_geothermal_cap^{phi}) + exp({ln_alpha_biomass=2})*(ln_biomass_cap^{phi}) + exp({ln_alpha_nuclear=2})*(ln_nuclear_cap^{phi}) + exp({ln_alpha_hydro=2})*(ln_hydro_cap^{phi}) + exp({ln_alpha_fossil=2})*(ln_fossil_cap^{phi}))^(1/{phi=1}))'

In [99]:
x = 'solar'
'(ln_all_sec_pr = {{c=2}} + ({0})^(1/{{phi=1}}))'.format(
    ' + '.join(['exp({{ln_alpha_{0}=-10}})*(ln_{0}_gen_hat^{{phi}})'.format(x) for x in sources]))

'(ln_all_sec_pr = {c=2} + (exp({ln_alpha_solar=-10})*(ln_solar_gen_hat^{phi}) + exp({ln_alpha_wind=-10})*(ln_wind_gen_hat^{phi}) + exp({ln_alpha_geothermal=-10})*(ln_geothermal_gen_hat^{phi}) + exp({ln_alpha_biomass=-10})*(ln_biomass_gen_hat^{phi}) + exp({ln_alpha_nuclear=-10})*(ln_nuclear_gen_hat^{phi}) + exp({ln_alpha_hydro=-10})*(ln_hydro_gen_hat^{phi}) + exp({ln_alpha_fossil=-10})*(ln_fossil_gen_hat^{phi}))^(1/{phi=1}))'

In [100]:
'(ln_all_sec_gen = {{c=1}} + ({0})^(1/{{phi=1}}))'.format(
    ' + '.join(['exp({{ln_alpha_{0}=2}})*(ln_{0}_gen_hat^{{phi}})'.format(x) for x in sources]))

'(ln_all_sec_gen = {c=1} + (exp({ln_alpha_solar=2})*(ln_solar_gen_hat^{phi}) + exp({ln_alpha_wind=2})*(ln_wind_gen_hat^{phi}) + exp({ln_alpha_geothermal=2})*(ln_geothermal_gen_hat^{phi}) + exp({ln_alpha_biomass=2})*(ln_biomass_gen_hat^{phi}) + exp({ln_alpha_nuclear=2})*(ln_nuclear_gen_hat^{phi}) + exp({ln_alpha_hydro=2})*(ln_hydro_gen_hat^{phi}) + exp({ln_alpha_fossil=2})*(ln_fossil_gen_hat^{phi}))^(1/{phi=1}))'